In [ ]:
# Instalar paquetes si es necesario
!pip install yfinance --quiet
!pip install matplotlib ipywidgets --quiet

# Importaciones
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output

# Modelos simulados que predicen hacia adelante
def modelo_lstm_adelante(data, dias):
    return [data[-1] + i * np.random.normal(0.5, 0.3) for i in range(1, dias+1)]

def modelo_arima_adelante(data, dias):
    return [data[-1] + np.cumsum(np.random.normal(0, 0.5, dias))[i] for i in range(dias)]

def modelo_rf_adelante(data, dias):
    return [data[-1] + np.random.normal(0, 2) for _ in range(dias)]

modelos = {
    'LSTM': modelo_lstm_adelante,
    'ARIMA': modelo_arima_adelante,
    'Random Forest': modelo_rf_adelante,
}

# Widgets de la interfaz
modelo_dropdown = widgets.Dropdown(
    options=list(modelos.keys()),
    value='LSTM',
    description='Modelo:',
)

fecha_inicio_picker = widgets.DatePicker(
    description='Fecha inicio:',
    value=datetime.now().date() - timedelta(days=90)
)

dias_slider = widgets.IntSlider(
    value=15,
    min=5,
    max=60,
    step=5,
    description='Días a predecir:',
)

boton_actualizar = widgets.Button(
    description='Actualizar gráfico',
    button_style='success',
    tooltip='Haz clic para actualizar el gráfico',
)

# Área de salida exclusiva para el gráfico
salida_grafico = widgets.Output()

# Función para actualizar el gráfico
def actualizar_grafico(b):
    with salida_grafico:
        salida_grafico.clear_output(wait=True)

        modelo_nombre = modelo_dropdown.value
        fecha_inicio = fecha_inicio_picker.value
        dias_pred = dias_slider.value
        fecha_fin = datetime.now().date()

        # Descargar últimos datos reales
        df = yf.download('AAPL', start=fecha_inicio, end=fecha_fin)
        if df.empty or len(df) < 2:
            print("No hay suficientes datos disponibles.")
            return

        reales = df['Close'].values
        fechas_reales = df.index

        # Predecir hacia el futuro desde el último punto
        modelo = modelos[modelo_nombre]
        predicho = modelo(reales, dias_pred)

        fechas_pred = [fechas_reales[-1] + timedelta(days=i) for i in range(1, dias_pred + 1)]

        # Graficar
        plt.figure(figsize=(12, 5))
        plt.plot(fechas_reales, reales, label='Real', linewidth=2)
        plt.plot(fechas_pred, predicho, label=f'Predicción ({modelo_nombre})', linestyle='--')
        plt.title(f'Predicción futura de {dias_pred} días con modelo {modelo_nombre}')
        plt.xlabel('Fecha')
        plt.ylabel('Precio')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

# Conectar botón
boton_actualizar.on_click(actualizar_grafico)

# Mostrar UI
ui = widgets.VBox([modelo_dropdown, fecha_inicio_picker, dias_slider, boton_actualizar, salida_grafico])
display(ui)
